In [ ]:
# 📊 Evaluación de Red Neuronal en Imágenes Histopatológicas

# 🔧 Cargar librerías
import os
import sys
sys.path.append("../src")

import yaml
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

import evaluate
import utils

In [ ]:
# 📁 Cargar configuración
with open("../config.yaml", "r") as f:
    config = yaml.safe_load(f)

model_path = config["paths"]["model"]
data_path = config["paths"]["prepared_data"]
batch_size = config["training"]["batch_size"]
img_height = config["image"]["height"]
img_width = config["image"]["width"]

In [ ]:
# 🔁 Cargar datos de validación (sin shuffle)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

val_generator = test_datagen.flow_from_directory(
    os.path.join(data_path, "test"),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=False
)

In [ ]:
# 💾 Cargar modelo entrenado
model = load_model(model_path)

In [ ]:
# 🔍 Evaluar rendimiento
loss, acc = model.evaluate(val_generator)
print(f"🔹 Pérdida (val_loss): {loss:.4f}")
print(f"🔹 Precisión (val_accuracy): {acc:.4f}")

In [ ]:
# 🎯 Predicciones y métricas
y_true = val_generator.classes
y_prob = model.predict(val_generator)

threshold = config.get("evaluation", {}).get("threshold", 0.5)
y_pred = (y_prob > threshold).astype(int).flatten()

#y_pred = (y_prob > 0.5).astype(int).flatten()

# 📉 Reporte de métricas
print("\n📋 Reporte de clasificación:\n")
print(classification_report(y_true, y_pred))

# 📚 Matriz de confusión
utils.show_confusion_matrix(y_true, y_pred)

In [ ]:
# 📈 Distribución de predicciones
evaluate.plot_prediction_distribution(y_true, y_prob)
evaluate.plot_f1_vs_threshold(y_true, y_prob, modelo="Red Neuronal")

In [ ]:
# 📌 Comentarios pedagógicos

# - A diferencia de datasets tabulares, aquí las predicciones se basan en características visuales complejas.
# - La elección del umbral de decisión (default=0.5) puede ser optimizada según el problema (por ejemplo, minimizar falsos negativos).
# - En proyectos médicos, es importante revisar visualmente algunas imágenes clasificadas erróneamente.

In [ ]:
# 🖼️ Mostrar imágenes mal clasificadas
evaluate.show_misclassified_images(val_generator, model)